In [1]:
import tensorflow as tf
from tensorflow import keras
print('tf.__version__ : ', tf.__version__)
print('keras.__version__ : ', keras.__version__)

tf.__version__ :  1.13.1
keras.__version__ :  2.2.4-tf


# Vocab loader

In [14]:
import numpy as np

def vocab_unpack(vocab):
    return vocab['idx2word'], vocab['word2idx'], vocab['idx2char'], vocab['char2idx']

vocab_mapping = np.load('data/ptb/vocab.npz')
idx2word, word2idx, idx2char, char2idx = vocab_unpack(vocab_mapping)

In [73]:
idx2word[100]
# word2idx.tolist()['hope']

'federal'

# Trained Model (with 1 epoch)

In [3]:
from six.moves import cPickle as pickle
from model.LSTMCNN import LSTMCNN

opt = pickle.load(open('cv/char-large.pkl', "rb"))
model = LSTMCNN(opt)
model.load_weights('cv/char-large.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
chars (InputLayer)              (20, 35, 21)         0                                            
__________________________________________________________________________________________________
chars_embedding (Embedding)     (20, 35, 21, 15)     765         chars[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (20, 35, 21, 50)     800         chars_embedding[0][0]            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)    

In [74]:
def most_similar(emb_layer, pos_word_idxs, neg_word_idxs=[], top_n=10):
    weights = emb_layer.weights[0]

    mean = []
    for idx in pos_word_idxs:
        mean.append(weights.value()[idx, :])

    for idx in neg_word_idxs:
        mean.append(weights.value()[idx, :] * -1)

    mean = tf.reduce_mean(mean, 0)

    dists = tf.tensordot(weights, mean, 1)
    best = tf.math.top_k(dists, top_n)

    # Mask words used as pos or neg
    mask = []
    for v in set(pos_word_idxs + neg_word_idxs):
        mask.append(tf.cast(tf.equal(best.indices, v), tf.int8))
    mask = tf.less(tf.reduce_sum(mask, 0), 1)

    return tf.boolean_mask(best.indices, mask), tf.boolean_mask(best.values, mask)

idxs, vals = most_similar(model.layers[-1], [100])

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    idxs = sess.run(idxs)
    vals = sess.run(vals)
    print(idxs)
    print(vals)

[442 240 530 163 423  76 620 632 114]
[0.05194158 0.04548101 0.04536441 0.04483165 0.04071609 0.04059325
 0.04024662 0.04014509 0.03893768]


In [75]:
for idx in idxs:
    print(idx2word[idx])

demand
high
named
program
chicago
two
below
warner
exchange
